In [29]:
import os.path
import numpy as np
import pandas as pd
import torch
import pickle
import subprocess
from matplotlib import pyplot as plt
from Bio import SeqIO
from importlib import reload
import warnings

from scipy.stats import mode
from sklearn.neighbors import KernelDensity
from skimage.graph import route_through_array
from scipy.spatial import cKDTree
from Bio import Entrez, SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import os

In [11]:
GT1_proteins = pd.read_csv('/Users/dahala/GitHub/VAE-enzymes/random_data/GT1.txt', sep='\t', header=None)
GT1_proteins.columns = ['type', 'organism', 'ID', 'Accession', 'database']
GT1_proteins = GT1_proteins.drop_duplicates(subset='Accession')
GT1_prot_ncbi = GT1_proteins[GT1_proteins['database'] == 'ncbi']


In [15]:
Entrez.email = "dahala@dtu.com"

def fetch_protein_sequence(accession):
    handle = Entrez.efetch(db="protein", id=accession, rettype="fasta", retmode="text")
    record = SeqIO.read(handle, "fasta")
    handle.close()
    return record

# Example usage:
acc = "BCM90193.1"  # Replace with your own accession number
protein_record = fetch_protein_sequence(acc)
print(protein_record.id)
print(protein_record.description)
print(protein_record.seq)

BCM90193.1
BCM90193.1 O-mycaminosyltylonolide 6-deoxyallosyltransferase [Abditibacteriota bacterium]
MRESALLKTLKILFSTFGSLGDIHPYVAIALEAKRRGHTPVIATSERYREKIAAQNIEFRPVAPDLPPEDEFAGLAKQVMNEKDGPRFLFEEILGPSIRTQYADLLAASEDVDLLISHPAAQTGPLVARKTGKKWLSSVLSPISLWSRRDPCVPPTLPHLDWLRVLGPLWGRIQVEAGKAATKKWVVGIEQLREEQNIEFAGHPMFGGQFSPFGTLALFSRHFCAPQPDWPQNTTATGFCFYDAVGLKSNSQPESSDWRAWMQNGSPPVVIGLGSAAVHDGAAIWDAAVRASRRDNVRVVLLTAGTFETAEENVLCVPYAPYSEIFPLARHVYHQGGVGTTAQALRAGVRQVIMPFAHDQSDNAARIQRLGVGRWMRRRELPSLNLKMTRSAWELQRFERAREIGELIRAENGPQAACESIERVGAM


In [24]:
def fetch_multiple(accessions):
    ids = ",".join(accessions)
    handle = Entrez.efetch(db="protein", id=ids, rettype="fasta", retmode="text")
    records = list(SeqIO.parse(handle, "fasta"))
    handle.close()
    return records

records = fetch_multiple(GT1_prot_ncbi.Accession.tolist())

In [34]:
def download_protein_sequences_to_fasta(accession_list, output_fasta, failed_log="failed_accessions.txt", batch_size=5000):
    all_sequences = []
    failed_accessions = []

    for i in range(0, len(accession_list), batch_size):
        batch = accession_list[i:i + batch_size]
        print(f"⏳ Fetching batch {i // batch_size + 1}: {len(batch)} accessions")

        try:
            ids = ",".join(batch)
            handle = Entrez.efetch(db="protein", id=ids, rettype="fasta", retmode="text")
            records = list(SeqIO.parse(handle, "fasta"))
            handle.close()
        except Exception as e:
            print(f"✗ Failed batch {i // batch_size + 1}: {e}")
            failed_accessions.extend(batch)
            continue

        fetched_ids = {record.id.split("|")[0] for record in records}
        all_sequences.extend(records)

        # Find any missing accessions in this batch
        missing = [acc for acc in batch if acc not in fetched_ids]
        if missing:
            print(f"⚠ {len(missing)} accessions not found in batch {i // batch_size + 1}")
            failed_accessions.extend(missing)

    # Write results
    if all_sequences:
        SeqIO.write(all_sequences, output_fasta, "fasta")
        print(f"\n✔ Saved {len(all_sequences)} sequences to {output_fasta}")

    if failed_accessions:
        with open(failed_log, "w") as f:
            for acc in failed_accessions:
                f.write(acc + "\n")
        print(f"⚠ {len(failed_accessions)} accessions failed. Saved to {failed_log}")

In [36]:
download_protein_sequences_to_fasta(GT1_prot_ncbi.Accession.tolist(), '../../random_data/GT1_filtered.fasta', failed_log="../../random_data/GT1_failed_accessions.txt",batch_size=9000)

⏳ Fetching batch 1: 9000 accessions
⚠ 2 accessions not found in batch 1
⏳ Fetching batch 2: 9000 accessions
⚠ 2 accessions not found in batch 2
⏳ Fetching batch 3: 9000 accessions
⚠ 2 accessions not found in batch 3
⏳ Fetching batch 4: 9000 accessions
⚠ 3 accessions not found in batch 4
⏳ Fetching batch 5: 9000 accessions
⏳ Fetching batch 6: 9000 accessions
⏳ Fetching batch 7: 3400 accessions

✔ Saved 57395 sequences to ../../random_data/GT1_filtered.fasta
⚠ 9 accessions failed. Saved to ../../random_data/GT1_failed_accessions.txt
